# Bonus Analysis

This notebook covers the bonus tasks:
1.  **Embedding Visualization**: Using UMAP to visualize sentence embeddings.
2.  **LLM Few-Shot Classification**: Using GPT-4 (or similar) for sentiment analysis via prompting.

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import umap
from sentence_transformers import SentenceTransformer

# Add src to path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from data_loader import load_and_split_data

## 1. Embedding Visualization (UMAP)

In [ ]:
splits = load_and_split_data()
test_df = splits['test']

# Generate embeddings for test set
model = SentenceTransformer("all-mpnet-base-v2")
embeddings = model.encode(test_df['sentence'].tolist(), show_progress_bar=True)

# Reduce dimensionality with UMAP
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
embedding_2d = reducer.fit_transform(embeddings)

# Plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(embedding_2d[:, 0], embedding_2d[:, 1], c=test_df['label'], cmap='viridis', s=10, alpha=0.7)
plt.legend(handles=scatter.legend_elements()[0], labels=['Negative', 'Neutral', 'Positive'])
plt.title('UMAP Visualization of Sentence Embeddings (Test Set)')
plt.show()

## 2. LLM Few-Shot Classification

**Note**: This requires an OpenAI API key. Please set `OPENAI_API_KEY` environment variable.

In [ ]:
import os
import openai
from sklearn.metrics import accuracy_score

api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    client = openai.OpenAI(api_key=api_key)
    
    def classify_with_llm(sentence):
        prompt = f"""
        You are a financial sentiment analysis assistant.
        Classify the sentiment of the following financial news sentence as Positive, Negative, or Neutral.
        Provide the label only.
        
        Examples:
        Sentence: "The company's profits surged by 50% this quarter."
        Sentiment: Positive
        
        Sentence: "The CEO resigned amidst a major scandal."
        Sentiment: Negative
        
        Sentence: "The company announced a new product line."
        Sentiment: Neutral
        
        Sentence: "{sentence}"
        Sentiment:
        """
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message.content.strip()

    # Run on a small subset of test data
    subset_test = test_df.head(20).copy()
    subset_test['llm_pred'] = subset_test['sentence'].apply(classify_with_llm)
    
    # Map text labels to integers if needed, or compare text
    label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
    subset_test['llm_pred_int'] = subset_test['llm_pred'].map(label_map)
    
    acc = accuracy_score(subset_test['label'], subset_test['llm_pred_int'])
    print(f"LLM Few-Shot Accuracy (on 20 samples): {acc:.4f}")
    
else:
    print("OPENAI_API_KEY not found. Skipping LLM classification.")